## [Salesforce Rest API](https://github.com/cghall/force-retrieve)

In [33]:
import io
import sys
import os
# import os.path
# from pathlib import Path
from os.path import expanduser as ospath
import numpy as np
import pandas as pd
from collections import OrderedDict
import pytz
import datetime
from simple_salesforce import Salesforce
from salesforce_reporting import Connection, ReportParser
excel_path = "~/Four Winds Interactive/Marketing - Documents/s_data/"            # directory - case sensistve
excel_file = "pardot.xlsx"                                                       # excel file - case sensistve

In [34]:
passfile = os.path.expanduser(os.path.join("~/", ".sfdc"))
with open(passfile) as f:       # read in authentication params
    uname, spass, stoken, ppass, ptoken = [x.strip("\n") for x in f.readlines()]
sf = Salesforce(username=uname, password=spass, security_token=stoken)      # authenticate
end = datetime.datetime.now(pytz.UTC)                                       # salesforce API requires UTC
print("@ ", end)

@  2019-06-23 16:41:47.557563+00:00


In [35]:
soql_opty = "queries/q_opty.sql"                                  # stored query
sheet_id_opty = "id_opty"                                         # sheet - case sensistve
with open(soql_opty, 'r') as file:                                # get opty soql query from file
    soql_opty = file.read().replace('\n','')                      # remove line breaks
    soql_opty = soql_opty.replace('\t','')                        # remove tabs
opty_id = pd.read_excel(excel_path + excel_file, sheet_id_opty)   # read ids from excel file
opty_id = tuple(list(opty_id['x18ContactID']))                    # convert dataframe column to list then tuple
opty_id = "','".join(opty_id)                                     # convert tuple to comma sep string
soql_opty = soql_opty + "'" + opty_id + "')"
q_opty = sf.query(soql_opty)
records = [dict(
      IndVert=rec['Account']['Industry_Vertical__c'],
         Name=rec['Account']['Name'],
     optysAct=rec['Account']['of_Active_Opps__c'],
     optyss30=rec['Account']['of_Opps_Created_Last_30_Days__c'],
     optyssYr=rec['Account']['of_Opps_Created_this_Calendar_Year__c'],
     x18actid=rec['Account']['X18_Digit_ID__c']) 
    for rec in q_opty['records']]
df_opty = pd.DataFrame(records)
df_opty.to_csv(excel_path + 'p_opty.csv')

In [36]:
soql_lead = "queries/q_lead.sql"                                   # stored query
sheet_id_lead = "email_lead"                                       # sheet - case sensistve
with open(soql_lead, 'r') as file:                                 # get lead soql query from file
    soql_lead = file.read().replace('\n','')                       # remove line breaks
    soql_lead = soql_lead.replace('\t','')                         # remove tabs
lead_email = pd.read_excel(excel_path + excel_file, sheet_id_lead) # read ids from excel file
lead_email = tuple(list(lead_email['Email']))                      # dataframe column to list to tuple
lead_email = "','".join(lead_email)                                # tuple to comma sep string
soql_lead = soql_lead + "'" + lead_email + "')"
q_lead = sf.query(soql_lead)
records = [dict(
           IndustryVertical=rec['Industry_Vertical__c'],
                      Email=rec['Email'],
                 LeadSource=rec['LeadSource'],
                    Company=rec['Company'],
                CreatedDate=rec['CreatedDate'],
                   LeadType=rec['Lead_Type__c'],
          UnqualifiedReason=rec['Unqualified_Reason__c'],
                         Id=rec['Id'],
         ConvertedAccountId=rec['ConvertedAccountId'],
     ConvertedOpportunityId=rec['ConvertedOpportunityId'],
         ConvertedContactId=rec['ConvertedContactId'])
    for rec in q_lead['records']]
df_lead = pd.DataFrame(records)
df_lead.to_csv(excel_path + 'p_lead.csv')

### [DataFrames](https://pandas.pydata.org)
  
```python
DataFrame.dtypes                        # column labels & data types
DataFrame.head(n)                       # return first n rows
DataFrame.columns                       # column labels
DataFrame.size                          # number (int) elements
DataFrame.memory_usage([index, deep])   # each column memory (bytes)
DataFrame.empty                         # empty
DataFrame.is_copy                       # return copy
DataFrame.read_csv('data/15Pct.csv')    # read in CSV file
DataFrame.to_csv ('opty.csv')           # write to csv file
DataFrame.pd.read_excel('xlsxfile', 'Sheet1') # read excel file
DataFrame.ExcelWriter('xlsxfile')  
DataFrame.to_excel('writer', 'Sheet1')  
writer.save()  
frame.to_excel('examples/ex2.xlsx')  
ex_out_path = "~/Four Winds Interactive/Marketing - Documents/s_data/"
ex_out_file = "pardotout.xlsx"
ex_path = os.path.join(ex_out_path, ex_file)
writer = pd.ExcelWriter(ospath(ex_in_path)+ex_out_file, engine = 'xlsxwriter')
df_opty.to_excel(writer)
writer.save()    
```

### [Salesforce get functions](https://developer.salesforce.com/docs/api-explorer)

Create file /root/.sfdc with credentials as:  
username  
password  
token  
uid
instance

authenticate 
```python
from salesforce_reporting import Connection, ReportParser
with open('.sfdc') as f:
    username, password, token, uid, instance = [x.strip("\n") for x in f.readlines()]
sf = Salesforce(username=username, password=password, security_token=token, sandbox=False)
sf = Salesforce(username=uname, password=spass, security_token=stoken)          # authenticate
end = datetime.datetime.now(pytz.UTC)                                           # salesforce API requires UTC
# print("Session ID:", session_id)
# print("Instance:", sf_instance)
# print("Version:", sf_version)
print("@ ", end)
```

pull report 
``` python
report = sf.get_report(_Customer_Cloud_T3)
parser = salesforce_reporting.ReportParser(report)
items = parser.records()
labels = parser._get_field_labels()
headers = [labels[i] for i in labels]
items.insert(0, headers)
df0 = pd.DataFrame(items)
header = df0.iloc[0]
df0 = df0.drop([0])
df0.columns = header
```

query
```python
ContactDict = sf.query("SELECT Company, CreatedDate, Email, FirstName, Industry, LastName, LeadSource, Title FROM Lead ORDER BY CreatedDate DESC NULLS FIRST LIMIT 50")

opportunity_amounts = sf.query("SELECT Id, Probability, StageName, Amount, Owner.Name FROM Opportunity WHERE AMOUNT < 10000")

amounts = [x['Amount'] for x in opportunity_amounts['records']]
owners = [x['Owner']['Name'] for x in opportunity_amounts['records']]
hist1 = go.Histogram(x=amounts)
py.iplot([hist1], filename='salesforce/opportunity-probability-histogram')

sf.Contact.updated(end - datetime.timedelta(days=10), end)       # contacts updated last 10d
contact = sf.Contact.get('003e0000003GuNXAA0')                   # 18 digit id
metadata = sf.Contact.metadata()
describe = sf.Contact.describe()
```

read into dataframe
```python
df = pd.DataFrame(ContactDict['records'])
df.head(10)
Age = df['Age__c']
Age.head(10)
agePlot = Age.hist(bins=10)
```